# TABLE OF CONTENTS:
---
* [Setup](#Setup)
    * [Connect to Workspace](#Connect-to-Workspace)
* [Data](#Data)
    * [Download Data](#Download-Data)
    * [Explore Data](#Explore-Data)
    * [Upload Data](#Upload-Data)
    * [Create and Register AML Dataset](#Create-and-Register-AML-Dataset)
* [Compute Target](#Compute-Target)
* [Training Artifacts](#Training-Artifacts)
* [Training Environment](#Development-Environment)
* [Compute Target](#Compute-Target)
* [Development Environment](#Development-Environment)
* [Experiment & Run Configuration](#Experiment-&-Run-Configuration)
    * [Option 1: Normal Script Run](#Option-1:-Normal-Script-Run)
    * [Option 2: Hyperdrive Run](#Option-2:-Hyperdrive-Run)
* [Run Monitoring](#Run-Monitoring)
* [Model Registration](#Model-Registration)
    * [Model Download](#Model-Download)
* [Resource Clean Up](#Resource-Clean-Up)
---

# Setup

Append parent directory to sys path to be able to import created modules from src directory.

In [ ]:
import sys
sys.path.append(os.path.dirname(os.path.abspath("")))

In [ ]:
# Import libraries
import azureml.core
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
import scipy.io
import torch
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import uuid

from azureml.core.authentication import MsiAuthentication
from azureml.core import Dataset, Environment, Experiment, Keyvault, Model, ScriptRunConfig, Workspace
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.model import InferenceConfig 
from azureml.train.hyperdrive import BanditPolicy, HyperDriveConfig, PrimaryMetricGoal, RandomParameterSampling
from azureml.train.hyperdrive import choice, uniform
from azureml.widgets import RunDetails
from torchvision import datasets

# Import created modules
from src.training.data_utils import download_data, load_data, imshow

print(f"azureml.core version: {azureml.core.VERSION}")

Automatically reload modules when changes are made.

In [ ]:
%load_ext autoreload
%autoreload 2

Create a training directory. This directory will contain all artifacts needed for model training. For AML remote training this directory will be copied to the remote compute at runtime.

In [ ]:
training_folder = os.path.join(os.getcwd(), "../src/training")
os.makedirs(training_folder, exist_ok=True)
print(f"Training folder {training_folder} has been created.")

Create a directory to store the training, validation and test data.

In [ ]:
data_folder = os.path.join(os.getcwd(), "../data")
os.makedirs(data_folder, exist_ok=True)
print(f"Data folder {data_folder} has been created.")

### Connect to Workspace

In order to connect and communicate with the Azure Machine Learning (AML) workspace, a workspace object needs to be instantiated using the Azure ML SDK.

In [ ]:
# Connect to the AML workspace.
# For alternative connection options see the aml_snippets directory.
msi_auth = MsiAuthentication()

ws = Workspace(subscription_id="bf088f59-f015-4332-bd36-54b988be7c90",
               resource_group="amlbrikserg",
               workspace_name="amlbriksews",
               auth=msi_auth)

# Data

In this template, a dataset provided in Azure Open Data Storage (https://azureopendatastorage.blob.core.windows.net/testpublic/temp/fowl_data.zip) is used to build a binary classification model that can classify chickens and turkeys.

### Download Data

Create a script with utility functions to download, load and display the data (this script already exists in the src folder).

In [ ]:
%%writefile $training_folder/data_utils.py
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
import torch
import urllib

from PIL import Image
from torchvision import datasets, transforms
from zipfile import ZipFile


def download_data():
    """
    Download and extract the data needed for model training.
    :return data_dir: directory where the data is stored
    """
    
    # download data
    print("Downloading archive file...")
    archive_file = "../data/fowl_data.zip"
    download_url = "https://azureopendatastorage.blob.core.windows.net/testpublic/temp/fowl_data.zip"
    urllib.request.urlretrieve(download_url, filename=archive_file)

    # extract files
    with ZipFile(archive_file, "r") as zip:
        print("Extracting files...")
        zip.extractall("../data")
        print("Finished extracting!")
        data_dir = os.path.join("../data", zip.namelist()[0])

    # delete zip file
    os.remove(archive_file)
    return data_dir


def load_data(data_dir):
    """
    Load the train/val data.
    :return (dataloaders, dataset_sizes, class_names):
        dataloaders: dictionary containing pytorch train and validation dataloaders
        dataset_sizes: dictionary containing the size of the training and validation datasets
        class_names: list containing all class names
    """

    # Data augmentation and normalization for training
    # Just normalization for validation
    data_transforms = {
        "train": transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        "val": transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }

    image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                              data_transforms[x])
                      for x in ["train", "val"]}
    
    dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                                  shuffle=True, num_workers=4)
                   for x in ["train", "val"]}
    
    dataset_sizes = {x: len(image_datasets[x]) for x in ["train", "val"]}
    
    class_names = image_datasets["train"].classes

    return dataloaders, dataset_sizes, class_names


def imshow(img):
    """
    Unnormalize an image batch retrieved from a dataloader and plot the batch
    """
    
    img = img / 2 + 0.5 # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0))) # transpose dimensions from Pytorch format to default numpy format
    plt.show()
    
    
def preprocess_image(image_file):
    """
    Preprocess an input image.
    :param image_file: Path to the input image
    :return image.numpy(): preprocessed image as numpy array
    """
    
    data_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    image = Image.open(image_file)
    image = data_transforms(image).float()
    image = image.clone().detach()
    image = image.unsqueeze(0)
    
    return image.numpy()

Download the data from https://azureopendatastorage.blob.core.windows.net/testpublic/temp/fowl_data.zip to the AML Compute Instance / local compute.

In [ ]:
fowl_data_directory = download_data()

### Explore Data

Create dataloaders to load the data into memory.

In [ ]:
dataloaders, dataset_sizes, class_names = load_data(fowl_data_directory)

Display the first batch of 4 images.

In [ ]:
# Get some random validation images
dataiter = iter(dataloaders["val"])
images, labels = dataiter.next()

# Show images
imshow(torchvision.utils.make_grid(images))
# Print labels
print(" ".join("%11s" % class_names[labels[j]] for j in range(4)))

### Upload Data

Upload the data to the default AML datastore.

In [ ]:
datastore = ws.get_default_datastore()
datastore.upload(src_dir="../data/fowl_data", target_path="data/fowl_data", overwrite=True)

### Create and Register AML Dataset

Register the data as a dataset in the AML workspace.

In [ ]:
# Create a dataset object from datastore location
dataset = Dataset.File.from_files(path=(datastore, "data/fowl_data"))

In [ ]:
# Register the dataset
dataset = dataset.register(workspace=ws,
                           name="fowl-dataset",
                           description="fowl dataset containing training and validation data",
                           create_new_version=True)

# Compute Target

Create a remote compute target to run experiments on. The below code will first check whether a compute target with name `cluster_name` already exists and if it does, it will use that instead of creating a new one.

In [ ]:
# Choose a name for the CPU cluster
cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing cluster, use it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", # CPU
                                                           # vm_size='STANDARD_NC6', # GPU
                                                           max_nodes=4,
                                                           idle_seconds_before_scaledown=2400)
    
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

# Use get_status() to get a detailed status for the current cluster
print(compute_target.get_status().serialize())

# Training Artifacts

A training script is created in the aml_training folder. This script will be executed by the remote compute. The training script uses transfer learning to train a pretrained ResNet18 model on the fowl dataset.

In [ ]:
%%writefile $training_folder/train.py
# Copyright (C) 2017, PyTorch contributors
# Adapted from https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

# Import libraries
import argparse
import copy
import numpy as np
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import urllib
from azureml.core import Run
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
from zipfile import ZipFile

# Import own modules
from data_utils import load_data
# from model import Net

# Get AML Run object
run = Run.get_context()


def train_model(model: torchvision.models,
                criterion: torch.nn.modules.loss,
                optimizer: torch.optim,
                scheduler: torch.optim.lr_scheduler,
                num_epochs: int,
                dataloaders: dict,
                dataset_sizes: dict) -> torchvision.models:
    """
    Train the model and track training and validation loss and accuracy.
    :param model: pretrained model which will be trained further
    :param criterion: torch loss function
    :param optimizer: torch optimizer
    :param scheduler: torch learning rate scheduler
    :param num_epochs: number of epochs to train the model
    :param dataloaders: dictionary of torch dataloaders
    :param dataset_sizes: dictionary with lengths of the training and val set
    :return model: pretrained model with tuned final fully connected layer
    """
    
    # Leverage GPU if available
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    start_time = time.time()

    # Load in weights of model
    best_model_weights = copy.deepcopy(model.state_dict())
    
    # Initialize best_acc
    best_acc = 0.0

    for epoch in range(num_epochs):
        print("=" * 20)
        print(f"Epoch {epoch + 1}/{num_epochs}")
        print("=" * 20)

        # Each epoch has a training and validation phase
        for phase in ["train", "val"]:
            if phase == "train":
                model.train() # Set model to training mode
            else:
                model.eval() # Set model to evaluate mode

            running_loss = 0.0
            running_correct_preds = 0

            # Iterate over data
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                # Track history only if in training phase
                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward pass, gradient optimization and learning rate update
                    # only if in training phase
                    if phase == "train":
                        loss.backward()
                        optimizer.step()
                        scheduler.step() 

                # Calculate statistics
                running_loss += loss.item() * inputs.size(0)
                running_correct_preds += torch.sum(preds == labels.data)
                

            # Average loss and accuracy over examples
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_correct_preds.double() / dataset_sizes[phase]

            print(f"{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")
            
            # Log the epoch validation loss and accuracy to AML run
            if phase == "train":
                run.log("train_loss", np.float(epoch_loss))
                run.log("train_acc", np.float(epoch_acc))
                
            if phase == "val":
                run.log("val_loss", np.float(epoch_loss))
                run.log("val_acc", np.float(epoch_acc))

            # Deep copy the model
            if phase == "val" and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_weights = copy.deepcopy(model.state_dict())

                # Log the best val accuracy to AML run
                run.log("best_val_acc", np.float(best_acc))
            
            if phase == "train":
                print("-" * 20)

    time_elapsed = time.time() - start_time
    
    print(f"Training completed in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s.")
    print(f"Best Val Acc: {best_acc:4f}")
          
    # Load best model weights
    model.load_state_dict(best_model_weights)
          
    return model


def fine_tune_model(num_epochs: int,
                    num_classes: int,
                    dataloaders: dict,
                    dataset_sizes: dict,
                    learning_rate: float,
                    momentum: float) -> torchvision.models:
    """
    Load a pretrained model and reset the final fully connected layer.
    :param num_epochs: number of epochs to train the model
    :param num_classes: number of target classes 
        (supports binary and multiclass classification)
    :param dataloaders: dictionary of torch dataloaders
    :param dataset_sizes: dictionary with lengths of the training and val set
    :param learning_rate: learning rate hyperparameter
    :param momentum: momentum hyperparameter
    :return model: pretrained model with tuned final fully connected layer
    """

    print("=" * 20)
    print("START TRAINING")
    print("=" * 20)
    
    # Log the hyperparameter metrics to the AML run
    run.log("lr", learning_rate)
    run.log("momentum", momentum)

    # Load pretrained model and reset final fully connected layer to have num_classes output neurons
    model_ft = models.resnet18(pretrained=True)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, num_classes)

    # Leverage GPU if available
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model_ft = model_ft.to(device)

    # Specify loss function
    criterion = nn.CrossEntropyLoss()

    # Create SGD optimizer to optimize all parameters
    optimizer_ft = optim.SGD(model_ft.parameters(),
                             lr=learning_rate,
                             momentum=momentum)
                            
    # Create scheduler to decay LR by a factor of 0.1 every 7 epochs
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft,
                                           step_size=7,
                                           gamma=0.1)
    
    # Start model training
    model = train_model(model_ft, criterion, optimizer_ft,
                        exp_lr_scheduler, num_epochs, dataloaders,
                        dataset_sizes)

    return model


def main():
    
    print("Torch version:", torch.__version__)
    
    # Retrieve command-line arguments
    parser = argparse.ArgumentParser()
    parser.add_argument("--data_path", type=str, help="Path where the images are stored")
    parser.add_argument("--num_epochs", type=int, default=25, help="Number of epochs to train")
    parser.add_argument("--output_dir", type=str, help="Output directory")
    parser.add_argument("--learning_rate", type=float, default=0.001, help="Learning rate")
    parser.add_argument("--momentum", type=float, default=0.9, help="Momentum")
    args = parser.parse_args()
          
    # Load training and validation data
    dataloaders, dataset_sizes, class_names = load_data(args.data_path)
        
    # Train the model
    model = fine_tune_model(num_epochs=args.num_epochs,
                            num_classes=len(class_names),
                            dataloaders=dataloaders,
                            dataset_sizes=dataset_sizes,
                            learning_rate=args.learning_rate,
                            momentum=args.momentum)
    
    # Save the model
    os.makedirs(args.output_dir, exist_ok=True)
    torch.save(model, os.path.join(args.output_dir, "model.pt"))
    print("=" * 20)
    print(f"Model saved in {args.output_dir}.")


if __name__ == "__main__":
    main()

In [ ]:
# %%writefile $training_folder/model.py
# import torch.nn as nn
# import torch.nn.functional as F


# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(3, 6, 5)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.conv2 = nn.Conv2d(6, 16, 5)
#         self.fc1 = nn.Linear(16 * 5 * 5, 120)
#         self.fc2 = nn.Linear(120, 84)
#         self.fc3 = nn.Linear(84, 10)

#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         x = x.view(-1, 16 * 5 * 5)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x

Run the training script locally for 2 epochs for debugging purposes.

In [ ]:
!python ../src/training/train.py --data_path ../data/fowl_data --num_epochs=2 --output_dir="../outputs" --learning_rate 0.01 --momentum 0.9

# Training Environment

Load the environment that has been registered as part of the **01_environment_creation** notebook and use it for remote training.

In [ ]:
env_name = "pytorch-aml-env"
env = Environment.get(workspace=ws, name=env_name)

# Experiment & Run Configuration

Now that the training artifacts are prepared, a model can be trained on the remote compute cluster. You can take advantage of Azure compute to leverage GPUs to cut down your training time. 

In [ ]:
# Create the experiment
experiment = Experiment(workspace=ws, 
                        name="fowl-pytorch")

experiment.tag("model_architecture", "transfer-learning with resnet18")

### Option 1: Normal Script Run

In [ ]:
# Set variable to identify run type for logic later in the notebook
run_type = "script_run"

# Create the script run configuration
src_config = ScriptRunConfig(source_directory="../src/training",
                             script="train.py",
                             compute_target=compute_target,
                             arguments=[
                                 "--data_path", dataset.as_named_input("input").as_mount(),
                                 "--num_epochs", 20,
                                 "--output_dir", "outputs",
                                 "--learning_rate", 0.01,
                                 "--momentum", 0.9])

src_config.run_config.environment = env

# Start the Script Run
run = experiment.submit(src_config)

### Option 2: Hyperdrive Run

Hyperparameters can be tuned using AML's hyperdrive capability.

The initial learning rate is tuned. The training script can contain a LR schedule to decay the learning rate every several epochs starting from that initial learning rate.

Random sampling is used to try different configuration sets of hyperparameters to maximize the primary metric, the best validation accuracy (best_val_acc).

An early termination policy is specified to early terminate poorly performing runs. The BanditPolicy is used, which will terminate any run that doesn't fall within the slack factor of the primary evaluation metric. In this template, this policy will be applied every epoch (since the best_val_acc metric is reported every epoch and evaluation_interval=1). The first policy evaluation will be delayed until after the first 10 epochs (delay_evaluation=10). 

In [ ]:
# Set variable to identify run type for logic later in the notebook
run_type = "hyperdrive_run"

param_sampling = RandomParameterSampling({
    "learning_rate": uniform(0.0005, 0.005),
    "momentum": uniform(0.9, 0.99)}
)

early_termination_policy = BanditPolicy(slack_factor=0.15, evaluation_interval=1, delay_evaluation=10)

# Create the script run configuration
src_config = ScriptRunConfig(source_directory="../src/training",
                             script="train.py",
                             compute_target=compute_target,
                             arguments=[
                                 "--data_path", dataset.as_named_input("input").as_mount(),
                                 "--num_epochs", 20,
                                 "--output_dir", "outputs",
                                 "--learning_rate", 0.01,
                                 "--momentum", 0.9])

src_config.run_config.environment = env

hyperdrive_config = HyperDriveConfig(run_config=src_config,
                                     hyperparameter_sampling=param_sampling, 
                                     policy=early_termination_policy,
                                     primary_metric_name="best_val_acc",
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=4,
                                     max_concurrent_runs=2)

# Start the Hyperdrive Run
run = experiment.submit(hyperdrive_config)

# Run Monitoring

In [ ]:
RunDetails(run).show()

In [ ]:
# Get portal URL
run.get_portal_url()

In [ ]:
run.wait_for_completion(show_output=False)

In [ ]:
# Retrieve best child run
if run_type == "script_run":
    best_child_run = run
elif run_type == "hyperdrive_run":
    best_child_run = run.get_best_run_by_primary_metric()

In [ ]:
# Check run metrics, details and file names
best_child_run_metrics = best_child_run.get_metrics()
best_child_run_details = best_child_run.get_details()
best_child_run_file_names = best_child_run.get_file_names()


print(best_child_run_metrics)
print("==========================")
print(best_child_run_details)
print("==========================")
print(best_child_run_file_names)

In [ ]:
print(f"Best Run is:")

if run_type == "script_run":
    print(f"Validation accuracy: {best_child_run_metrics['best_val_acc'][-1]}")
elif run_type == "hyperdrive_run":
    print(f"Validation accuracy: {best_child_run_metrics['best_val_acc']}")

print(f"Learning rate: {best_child_run_metrics['lr']}")
print(f"Momentum: {best_child_run_metrics['momentum']}")

# Model Registration

In [ ]:
model_path = "outputs/model.pt"

model = best_child_run.register_model(model_name="fowl-model",
                                      model_path=model_path,
                                      model_framework=Model.Framework.PYTORCH,
                                      description="fowl model")

print(model.name, model.id, model.version, sep="\t")

### Model Download

In [ ]:
download_model=True

if download_model:
    
    # Create directory
    outputs_folder = os.path.join(os.getcwd(), "../outputs")
    os.makedirs(outputs_folder, exist_ok=True)
    print(f"Outputs folder {outputs_folder} has been created.")
    
    # Download model artifact
    best_child_run.download_file(name=model_path, output_file_path="../outputs/model.pt")

# Resource Clean Up

In [ ]:
compute_target.delete()